In [0]:
catalog = "bronze"
schema = "mastercard_reporting"
table_name = dbutils.widgets.get("table_name")

In [0]:
df_full = (spark.read.
            format("csv").
            load(f"/Volumes/raw/{schema}/{table_name}", 
                 header=True, 
                 inferSchema=True
            )
        )

In [0]:
def table_exists(catalog, schema, table):
    count = (spark.sql(f"SHOW TABLES FROM {catalog}.{schema}")
                .filter(f"database = '{schema}' AND tableName = '{table}'")
                .count()
            )
    return count > 0

In [0]:
import delta

# Condition merge tables
def condition_table(table_name):
    if table_name == "cards":
        return "t1.card_number = t2.card_number"
    elif table_name == "cards_status":
        return "t1.card_number = t2.card_number and t1.card_status = t2.card_status"
    elif table_name == "cards_transactions":
        return "t1.transaction_id = t2.transaction_id"

if not table_exists(catalog, schema, table_name):
    
    print("Table is creating now...")
    (df_full
        .coalesce(1).
        write.
        format("delta").
        mode("overwrite").
        saveAsTable(f"{catalog}.{schema}.{table_name}")
    )
else:
    print("Table is inserting new ids...")
    deltaTable = delta.DeltaTable.forName(spark, tableOrViewName=f"{catalog}.{schema}.{table_name}")
    deltaTable.alias("t1").merge(
        df_full.alias("t2"),
        f"{condition_table(table_name)}"
    ).whenNotMatchedInsertAll().execute()